In [330]:
from serpapi import GoogleSearch
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import json
import os

In [355]:
#########################################################
# WARNING!                                              #
# Don't run with this API key unless 100% required      #
#########################################################

# API Key Store
# Travis: 7b49b198668e629ad2ea004850f238d18e56e6e6f3ed19a4d71430a670f4ba29
# Leon: a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada
# Konni: 

params = {
    "engine": "google_jobs",
    "q": "data scientist",
    "hl": "en",
    "api_key": "a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada",
    "gl": "uk",
    "location": "United Kingdom"
}

search = GoogleSearch(params)
results = search.get_dict()
jobs_results = results["jobs_results"]

https://serpapi.com/search


In [356]:
# Pretty Print the results
pprint(results)

{'chips': [{'options': [{'text': 'All'},
                        {'text': 'Data scientist', 'value': 'data scientist'},
                        {'text': 'Data science', 'value': 'data science'},
                        {'text': 'Manager', 'value': 'manager'},
                        {'text': 'Graduate', 'value': 'graduate'},
                        {'text': 'Consultant', 'value': 'consultant'},
                        {'text': 'Engineer', 'value': 'engineer'},
                        {'text': 'Student', 'value': 'student'},
                        {'text': 'Analyst', 'value': 'analyst'},
                        {'text': 'Apprenticeship', 'value': 'apprenticeship'},
                        {'text': 'Associate', 'value': 'associate'},
                        {'text': 'Cfo', 'value': 'cfo'},
                        {'text': 'Data analyst', 'value': 'data analyst'},
                        {'text': 'Insight analyst', 'value': 'insight analyst'},
                        {'text': 'Portfolio 

In [396]:
# Generate raw dataframe from scraped job listings
df_raw = pd.DataFrame.from_records(jobs_results)

In [397]:
## DATA CLEANING ##

# Remove unnecessary bracket in Location
df_raw['location'] = df_raw['location'].str.replace('\([^()]*\)', "", regex=True)

# Remove via keyword from via
df_raw['via'] = df_raw['via'].str.replace('via', '', regex=False)

# Remove new line and other unnecessary characters
df_raw = df_raw.replace(to_replace=r'\n\n|\n•|\n', value=' ', regex=True)

# Remove unncecessary columns from the dataframe
df_raw.drop(columns=['thumbnail', 'extensions', 'related_links'], inplace=True)

In [399]:
## EXTRACTION ##

## Qualifications, responsibilities, and benefits ##
qualifications = []
responsibilities = []
benefits = []
highlights = []
def extract_inner_df(x): 
    for highlight in x:
        column_title = highlight.get('title')
        column_items = highlight.get('items')
        highlights_selection(column_title, column_items)
    equalSeries()
    
def highlights_selection(title, items):
    match title:
        case 'Qualifications':
            qualifications.append(items)
        case 'Responsibilities':
            responsibilities.append(items)
        case 'Benefits':
            benefits.append(items)
        case None:
            highlights.append(items)
            
def equalSeries():
    lengths = [len(qualifications), len(responsibilities), len(benefits), len(highlights)]
    max_length = max(lengths)
    
    if len(qualifications) < max_length:
        qualifications.append(None)
    if len(responsibilities) < max_length:
        responsibilities.append(None)
    if len(benefits) < max_length:
        benefits.append(None)
    if len(highlights) < max_length:
        highlights.append(None)
        
def merging_highlights():
    df_raw.drop(columns=['job_highlights'], inplace=True)
    df_raw['qualifications'] = qualifications
    df_raw['responsibilities'] = responsibilities
    df_raw['benefits'] = benefits
    df_raw['highlights'] = highlights
    
    
# Expand and merge job insights into the raw_df
df_raw['job_highlights'].apply(extract_inner_df)
df_raw.head()
merging_highlights()


# Display
df_raw.head(10)

,title,company_name,location,via,description,detected_extensions,job_id,qualifications,responsibilities,benefits,highlights
0,"Data Scientist - £70,000 - Edinburgh Hybrid",In Technology Group Limited,Edinburgh,reed.co.uk,Data Scientist | Edinburgh We are a recruitmen...,"{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIMKjNz...,None,None,None,[Data Scientist | Edinburgh\n\nWe are a recrui...
1,Product Owner (Data Science Focus),be-it,Glasgow,Be-IT Ltd,Product Owner – Data Science Background Tech-F...,"{'posted_at': '1 day ago', 'schedule_type': 'F...",eyJqb2JfdGl0bGUiOiJQcm9kdWN0IE93bmVyIChEYXRhIF...,None,None,None,[Product Owner – Data Science Background Tech-...
2,IT Graduate - Data Scientist,Graduates into Health,England,Milkround,IT Graduate - Data Scientist Organisation: Uni...,{'posted_at': '8 days ago'},eyJqb2JfdGl0bGUiOiJJVCBHcmFkdWF0ZSAtIERhdGEgU2...,None,None,None,[IT Graduate - Data Scientist\n\nOrganisation:...
3,Data Scientist,MCS Group,Belfast,Nijobfinder,MCS group is proud to be working with one of t...,"{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,None,None,None,[MCS group is proud to be working with one of ...
4,Data Scientist - £50-70k,Amber Resourcing,Birmingham,LinkedIn,Are you interested in beginning your career at...,"{'posted_at': '21 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIMKjNT...,None,None,None,[Are you interested in beginning your career a...
5,Data Scientist,83zero Limited,Derby,Totaljobs,"Data Scientist Salary: £30,000-£40,000 Locatio...","{'posted_at': '1 day ago', 'schedule_type': 'F...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,None,None,None,"[Data Scientist\nSalary: £30,000-£40,000\nLoca..."
6,Data Scientist (Hybrid),Citi,Belfast,EFinancialCareers,Overview of Citi: Citi is a world-leading glob...,"{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoSHlicm...,None,None,None,[Overview of Citi:\nCiti is a world-leading gl...
7,Senior Data Scientist,Be-IT Resourcing,Renfrewshire,reed.co.uk,Do you want to be part of a team responsible f...,"{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,None,None,None,[Do you want to be part of a team responsible ...
8,Data Scientist,Enso Recruitment,Anywhere,NIJobs.com,Data Scientist (Healthcare) Permanent Belfast ...,"{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,None,None,None,[Data Scientist (Healthcare)\nPermanent\nBelfa...
9,Senior Data Scientist,Be-IT Resourcing Ltd,Edinburgh,CV-Library,Do you want to be part of a team responsible f...,"{'posted_at': '1 day ago', 'schedule_type': 'F...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,None,None,None,[Do you want to be part of a team responsible ...


In [360]:
def column_matching():
    try:
        df = df_raw[['title', 'company_name', 'location', 'via', 'description',
                'detected_extensions', 'highlights', 'responsibilities', 'qualifications', 
                 'benefits', 'job_id']]
        return df
    except:
        print("!Essential columns missing!")

def write_data(df):
    file_name = 'interim_data.csv'
    data_exists = os.path.isfile(file_name)
    if data_exists:
        df_raw.to_csv(file_name, mode='a', index=False, header=False)
    else:
        df_raw.to_csv(file_name, mode='w', index=False, header=True)
   


df_final = column_matching()
write_data(df_final)